# Utilisation de Mistral-7B

## Introduction

Dans ce notebook, nous allons utiliser Mistral-7B (https://huggingface.co/mistralai/Mistral-7B-v0.3) à partir de l'API Transformers d'Hugging Face. Les étapes suivantes seront suivies :

1. Installation des packages.
2. Importation des bibliothèques nécessaires.
3. Récupération du modèle Mistral-7B.
4. Prédiction à partir du modèle.
5. Sauvegarde du modèle sur le disque

## Installation des packages

In [1]:
! pip install transformers[sentencepiece] trl accelerate torch bitsandbytes peft datasets -qU

## Importation des bibliothèques nécessaires

In [3]:
# Pour télécharger les données
from datasets import load_dataset

# Pour avoir l'accès au Hub d'Hugging Face
from huggingface_hub import notebook_login

# Pour récupérer Mistral-7B, le tokenizer associé et ...
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Pour utiliser PyTorch
import torch

## Récupération du modèle Mistral-7B

Dans cette section, on télécharge/récupère le modèle de base Mistral-7B et le tokenizer associé.

On commence par se connecter à Hugging Face à partir d'un token qu'on récupère sur Hugging Face :

In [4]:
# Connexion à Hugging Face grâce à un token
notebook_login()

In [3]:
# TODO: À étudier
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.float
)

In [4]:
model_id = "mistralai/Mistral-7B-v0.3"

In [5]:
# Récupération de Mistral-7B
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map='auto',
    quantization_config=nf4_config,
    use_cache=False,
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
# Récupération du tokenizer associé
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.3")

# Problème: ValueError: Cannot instantiate this tokenizer from a slow version. If it's based on sentencepiece, make sure you have sentencepiece installed.
# Solution: pip install transformers[sentencepiece]

In [7]:
# TODO: À étudier
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

## Prédiction à partir du modèle

In [32]:
def generate_response(prompt, tokenizer, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
    
  print("Tokenization du prompt:")
  print(encoded_input)
  print("\n")
    
  print("Affichage des tokens:")
  print(tokenizer.tokenize(tokenizer.decode(encoded_input["input_ids"][0])))
  print("\n")
    
  model_inputs = encoded_input.to('cuda')
    
  print("Chargement dans cuda:")
  print(model_inputs)
  print("\n")

  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  print("IDs générés:")
  print(generated_ids)
  print("\n")

  decoded_output = tokenizer.batch_decode(generated_ids)

  print("Décodage des IDs générés:")
  print(decoded_output[0])
  print("\n")

  print("Suppression du prompt:")
  print(decoded_output[0].replace(prompt, ""))
  print("\n")

  return decoded_output[0].replace(prompt, "")

Exemple d'utilisation de la fonction `generate_response` :

In [29]:
# Prompt
prompt1 = "### Instruction:\nUse the provided input to create a response.\n\n### Input:\nCan I find information about SALOME platform ?\n\n### Response:"
prompt2 = "### Instruction:\nUse the provided input to create a response.\n\n### Input:\nWhat is the CEA Research Center website ?\n\n### Response:"

print("########################### Prompt 1 ###########################")
print("\n")
print(prompt1)
print("\n")

print("########################### Prompt 2 ###########################")
print("\n")
print(prompt2)
print("\n")

########################### Prompt 1 ###########################


### Instruction:
Use the provided input to create a response.

### Input:
Can I find information about SALOME platform ?

### Response:


########################### Prompt 2 ###########################


### Instruction:
Use the provided input to create a response.

### Input:
What is the CEA Research Center website ?

### Response:




In [33]:
# Réponse prédite par le modèle
toto = generate_response(prompt1, tokenizer, model)

Tokenization du prompt:
{'input_ids': tensor([[    1,  1542,  3901,  3880, 29515,   781,  9311,  1040,  4625,  3555,
          1066,  2999,  1032,  3667, 29491,   781,   781, 28100, 12000, 29515,
           781,  7093,  1083,  2068,  2639,  1452,  1086,  1854, 29530,  2342,
          5949,  2318,   781,   781, 28100, 12875, 29515]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


Affichage des tokens:
['<s>', '▁###', '▁Inst', 'ruction', ':', '<0x0A>', 'Use', '▁the', '▁provided', '▁input', '▁to', '▁create', '▁a', '▁response', '.', '<0x0A>', '<0x0A>', '###', '▁Input', ':', '<0x0A>', 'Can', '▁I', '▁find', '▁information', '▁about', '▁S', 'AL', 'O', 'ME', '▁platform', '▁?', '<0x0A>', '<0x0A>', '###', '▁Response', ':']


Chargement dans cuda:
{'input_ids': tensor([[    1,  1542,  3901,  3880, 29515,   781,  9311,  1040,  4625,  3555,
          1066,  2999,  1032,  3667, 29491,   781,   78

In [34]:
# Réponse prédite par le modèle
toto = generate_response(prompt2, tokenizer, model)

Tokenization du prompt:
{'input_ids': tensor([[    1,  1542,  3901,  3880, 29515,   781,  9311,  1040,  4625,  3555,
          1066,  2999,  1032,  3667, 29491,   781,   781, 28100, 12000, 29515,
           781,  3963,  1117,  1040,  1102, 19540,  8750,  6832,  5168,  2318,
           781,   781, 28100, 12875, 29515]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


Affichage des tokens:
['<s>', '▁###', '▁Inst', 'ruction', ':', '<0x0A>', 'Use', '▁the', '▁provided', '▁input', '▁to', '▁create', '▁a', '▁response', '.', '<0x0A>', '<0x0A>', '###', '▁Input', ':', '<0x0A>', 'What', '▁is', '▁the', '▁C', 'EA', '▁Research', '▁Center', '▁website', '▁?', '<0x0A>', '<0x0A>', '###', '▁Response', ':']


Chargement dans cuda:
{'input_ids': tensor([[    1,  1542,  3901,  3880, 29515,   781,  9311,  1040,  4625,  3555,
          1066,  2999,  1032,  3667, 29491,   781,   781, 28100, 12000, 29515,
         

## Sauvegarde du modèle sur le disque

In [37]:
# Spécification du répertoire de sauvegarde
save_directory_model = "../models/mistral7b_not_fine-tune"
save_directory_tokenizer = "../models/mistral7b_tokenizer_not_fine-tune"

# Sauvegarde du modèle non fine-tuné sur le disque
model.save_pretrained(save_directory_model)

# Sauvegarde du tokenizer non fine-tuné sur le disque
tokenizer.save_pretrained(save_directory_tokenizer)

('../models/mistral7b_tokenizer_not_fine-tune/tokenizer_config.json',
 '../models/mistral7b_tokenizer_not_fine-tune/special_tokens_map.json',
 '../models/mistral7b_tokenizer_not_fine-tune/tokenizer.model',
 '../models/mistral7b_tokenizer_not_fine-tune/added_tokens.json',
 '../models/mistral7b_tokenizer_not_fine-tune/tokenizer.json')